In [5]:
import sys
sys.path.append('..')
import constants.constants as const
import constants.file_handler_constants as fh
from constants.attraction_constants import *

from packages.attraction.Attraction import *
from packages.file_handler_package.file_handler import *

import os
import glob
import time
import pandas as pd
import ast
import json
import requests
import ast

In [6]:
path_to_res_merge_attraction_phuket = os.path.join(fh.STORE_FILL_ATTRACTION_TAG_SCORE, 'res_merge_attraction', 'res_merge_attraction_Phuket.csv')
merge_attraction_phuket_df = pd.read_csv(path_to_res_merge_attraction_phuket)

merge_attraction_phuket_df = merge_attraction_phuket_df.fillna('')

In [7]:
def create_payload(row: pd.Series) -> dict:

    openingHour_list_payload = [{ "day": key, "openingHour": val } for key, val in ast.literal_eval(row['openingHour']).items()]

    attractionTagFields_dict_payload = {cur_tag:row[cur_tag] for cur_tag in ATTRACTION_TAG_SCORE}

    payload = {
        "name": row['name'],
        "type": ast.literal_eval(row['type']),
        "description": row['description'],
        "latitude": row['latitude'],
        "longitude": row['longitude'],
        "imgPath": ast.literal_eval(row['imgPath']),
        "phone": row['phone'],
        "website": row['website'],
        "openingHour": openingHour_list_payload.copy(),
        "attractionTag": {
            "attractionTagFields": attractionTagFields_dict_payload.copy()
        },
        "location": {
            "address": row['address'],
            "province": row['province'],
            "province_code": row['province_code'],
            "district": row['district'],
            "district_code": row['district_code'],
            "subDistrict": row['subDistrict'],
            "sub_district_code": row['sub_district_code']
        },
        "rating": {
            "score": row['score'],
            "ratingCount": row['ratingCount']
        }
    }

    return payload.copy()

In [ ]:
for Idx, cur_row in merge_attraction_phuket_df.iterrows():

    if(Idx == 1):
        break

    cur_payload = create_payload(row=cur_row)

    try:
        API_ENDPOINT = f"{os.environ['TRIPWEAVER_API']}/api/attraction/create"
        post_request = requests.post(url=API_ENDPOINT, data=cur_payload)
        print(post_request.text)
        
    except Exception as e:
        print("POST api failed at --> ", cur_payload['name'])
        print(e)

{"message":"An error occured while insert data attraction SyntaxError: Unexpected token 'a', \"name=%E0%B9\"... is not valid JSON"}
